![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/taller_3/main/figs/taller-meca-aprendizaje%20no%20supervisado_banner%201169%20x%20200%20px%20-05.png)

# Caso-taller:  Recomendando el Blog de  Hernán Casciari


[Hernán Casciari](https://hernancasciari.com/#bio), es un escritor argentino, que escribe blog posts con cuentos e historias  relacionadas con el futbol, su vida, infancia, y relaciones familiares con toques de ficción. Este [blog](https://hernancasciari.com/blog/) es  tan interesantes que en 2005 fue premiado como “El mejor blog del mundo” por Deutsche Welle de Alemania.

El objetivo de este caso-taller es construir un sistema de recomendación basado en los contenidos de los posts utilizando similitud de las palabras usadas o temas de los cuentos.

## Instrucciones generales

1. Para desarrollar el *cuaderno*, primero debe descargarlo.

2. Para responder cada inciso deberá utilizar el espacio debidamente especificado.

3. La actividad será calificada sólo si sube el *cuaderno* de jupyter notebook con extensión `.ipynb` en la actividad designada como "entrega calificada por el personal".

4. El archivo entregado debe poder ser ejecutado localmente por el tutor. Sea cuidadoso con la especificación de la ubicación de los archivos de soporte, guarde la carpeta de datos en el mismo `path` de su cuaderno, por ejemplo: `data`.

## Desarrollo


### 1. Carga de datos

En la carpeta `data` se encuentran el archivo `blog_casciari.csv` con el título, la fecha de publicación, y el contenido de los cuentos publicados en el blog  de sr. Casciari. Cargue estos datos en su *cuaderno* y reporte brevemente el contenido de la base.
   

In [1]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 84.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
pip install -q pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 32.9 MB/s eta 0:00:00


In [5]:
# cargamos librerias
import pandas as pd
import numpy as np
import os

import re
from spellchecker import SpellChecker
import unicodedata

import spacy
from spacy.language import Language

import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
url = 'https://raw.githubusercontent.com/sebastian23escobar/Aprendizaje_no_supervisado/refs/heads/main/Taller3/data/blog_casciari.csv'

In [6]:
blog = pd.read_csv(url)
blog.head(10)

,titulo,fecha,cuento
0,El rincón blanco,1/11/08,De pronto yo estaba en el hogar donde pasé la ...
1,Mínimos avances en la cama,1/24/08,"Menos la cama, todo ha mejorado en este mundo...."
2,Don Marcos,2/19/08,"Dos veces, y no una, mi abuelo materno me ayud..."
3,Los dos rulfos,3/26/08,"A su regreso de México, mi amigo Comequechu no..."
4,La noticia no es el perro,4/15/08,"De repente, un video de You Tube recibe un mil..."
5,El milagro de los pueblos,4/22/08,Cuando nació la Nina no tuve ganas de escribir...
6,Ropa sucia,5/6/08,Ya de entrada caí mal parado. Vine al mundo ju...
7,La Última Gran Guerra del Hombre Chiquito,5/13/08,Años atrás mi tataranieto Woung comenzó a envi...
8,Instrucciones para crear mundos paralelos,5/20/08,"Descubrimos el truco por casualidad, en nuestr..."
9,Una década flaca,5/27/08,Una tarde de 1990 fui a tomar la leche a la ca...


### 2. Homogenización de textos

Para cumplir con el objetivo de generar recomendaciones en esta sección debe preparar los posts para poder ser utilizados en su sistema de recomendación. Para ello, "limpie" y "tokenize" cada uno de los cuentos, describiendo detalladamente los pasos que realizo y si transformó o eliminó ciertas palabras. Para asistirlo en la tarea he creado listas de *stopwords* que están disponibles en la carpeta `data`. En su procedimiento ilustre la limpieza con el cuento 'La venganza del metegol'. (En su limpieza recuerde que el objetivo es generar recomendaciones a partir de la similitud de las palabras o temas de los cuentos)

In [7]:
blog.info()
# no tenemos problemas con missing alues

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   titulo  520 non-null    object
 1   fecha   520 non-null    object
 2   cuento  520 non-null    object
dtypes: object(3)
memory usage: 12.3+ KB


In [8]:
spell = SpellChecker(language='es')

import requests

# Añadir palabras correctas al diccionario para evitar falsos positivos
url = "https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/es/es_50k.txt"
response = requests.get(url)
with open("vocab.txt", "wb") as f:
    f.write(response.content)


with open('vocab.txt', 'r', encoding='utf-8') as f:
    palabras_externas = [line.strip().split()[0] for line in f]

spell.word_frequency.load_words(palabras_externas)

def corregir_texto(texto):
    # Separar palabras y signos de puntuación
    tokens = re.findall(r'\w+|[^\w\s]', texto, re.UNICODE)

    corregidas = []
    for token in tokens:
        if token.isalpha():
            if token not in spell:
                correccion = spell.correction(token)
                corregidas.append(correccion if correccion is not None else token)
            else:
                corregidas.append(token)
        else:
            corregidas.append(token)  # Mantener puntuación tal cual

    # Reconstruir el texto respetando espacios entre palabras y no entre signos
    texto_corregido = ''
    for i, token in enumerate(corregidas):
        if i > 0 and token not in ',.!?':
            texto_corregido += ' '
        texto_corregido += token

    return texto_corregido

In [9]:
nlp = spacy.load("es_core_news_sm")

# Cargamos las stopwords extra
extra_stopwords = pd.read_csv('https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S6_clase_sistemas_recomendacion_II/main/data/extra_stopwords.csv', sep=',',header=None)
extra_stopwords.columns = ['stopwords']
extra_stopwords=set(extra_stopwords['stopwords'].to_list())

# Agregamos a nuestro modelo de SpaCy
nlp.Defaults.stop_words |= extra_stopwords

#print(nlp.Defaults.stop_words) # Mostrar la nueva lista de stopwords

In [10]:
# Lista de stopwords en español
stopwords_es = set(stopwords.words('spanish'))

def limpieza_texto(texto, lista_palabras=[]):
    # remover acentos
    texto_sin_acentos = ''.join(c for c in unicodedata.normalize('NFD', texto)
                              if unicodedata.category(c) != 'Mn')

    # remover signos de puntuación y simbolos adicionales
    texto_sin_puntuacion = re.sub(r'[^\w\s]', '', texto_sin_acentos)


    texto = texto_sin_puntuacion.lower()
    # remover numeros
    texto = re.sub(r'\d+', '', texto)

    tokens = texto.split()
    tokens_filtrados = [
        t for t in tokens
        if t not in stopwords_es and t not in lista_palabras
    ]

    return ' '.join(tokens_filtrados)

In [11]:
ejemplo = blog['cuento'][500]
modificacion1 = limpieza_texto(ejemplo)
modificacion2 = corregir_texto(modificacion1)
print(modificacion2)

nino preguntas dificiles pocas vienen cabeza traen respuesta incluida sol linterna grande luna balon quedado colgado noche lobo perro madre nino todas preguntas poesia entonces padres idiotas enviar escuela olvides sospechar pensar conviertes adolescente despojo mago telefono movil solo sabe preguntar estupideces sms poner vocales misterio vida grandes preguntas mueren adolescencia sexo ocupa toda memoria tambien futbol motos fundas telefonos perdi magia alli suerte mas tarde recupere precio sido enfermedad recuperar todas preguntas todas respuestas hace dano cabeza dano si hace quedas apelado suerte puedes disimular posible llegues cantador poeta arquitecto actor teatro bailarina si suerte mal dia matas padre encierran hospital consigues cargo politico agente inmobiliario embargo suerte dejas hacer preguntas nunca acabas buscar respuestas extrañas extrano ocupa casi mundo extrano motor da vida madre pario ir mas lejos senora extraña aqui hacia va humanidad venimos razon vida sido mas 

No hemos podido limpiar todo el DataFrame al tiempo. Proponemos dividirlo en 5 bloques de 104 observaciones cada uno y, así, limpiarlo

In [14]:
num_bloques = 5
tamano_bloque = 104

lista_dfs = [blog[i * tamano_bloque:(i + 1) * tamano_bloque] for i in range(num_bloques)]

In [18]:
import os

for i, df in enumerate(lista_dfs):
  nombre_archivo = f'datos/blog_bloque_{i + 1}.csv'
  df.to_csv(nombre_archivo, index=False)

OSError: Cannot save file into a non-existent directory: 'datos'

In [ ]:
blog['cuento_limpio'] = blog['cuento'].apply(lambda x: corregir_texto(limpieza_texto(x)))

### 3. Generando Recomendaciones

En esta sección nos interesa generar recomendaciones de cuentos en el blog a un usuario que leyó 'La venganza del metegol'. Para ello vamos a utilizar distintas estrategias.

#### 3.1. Recomendaciones basadas en contenidos

##### 3.1.1. Genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para el cuento 'La venganza del metegol' usando en la distancia de coseno donde el texto este vectorizado por `CountVectorizer`. Explique el procedimiento que realizó y como ordenó las recomendaciones.

In [ ]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis, y conclusiones)

##### 3.1.2. Genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para  el cuento 'La venganza del metegol' usando nuevamente la distancia de coseno, pero ahora vectorice el texto usando `TF-IDFVectorizer`. Explique el procedimiento que realizó y como ordenó las recomendaciones. Compare con los resultados del punto anterior y explique sus similitudes y/o diferencias.

In [ ]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis, y conclusiones)

##### 3.1.3. Genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para el cuento 'La venganza del metegol' usando el texto vectorizado por `TF-IDFVectorizer` y la correlación como medida de similitud. Explique el procedimiento que realizó y como ordenó las recomendaciones. Compare con los resultados de los puntos anteriores y explique sus similitudes y/o diferencias.

In [ ]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis, y conclusiones)

##### 3.2. Recomendaciones basadas en temas

Usando modelado de temas con LDA, encuentre los temas subyacentes en el blog. Explique como eligió el numero óptimo de temas. Utilizando el tema asignado al cuento 'La venganza del metegol' y la probabilidad de pertenecer a este tema genere 5 recomendaciones de más recomendada (1) a menos recomendada (5) para este cuento. Explique el procedimiento que realizó. Compare con los resultados encontrados anteriormente y explique sus similitudes y/o diferencias. (Esto puede tomar mucho tiempo y requerir mucha capacidad computacional, puede aprovechar los recursos de [Google Colab](https://colab.research.google.com/))


In [ ]:
# Utilice este espacio para escribir el código.

(Utilice este espacio para describir el procedimiento, análisis, y conclusiones)

### 4 Recomendaciones generales

De acuerdo con los resultados encontrados, en su opinión ¿qué procedimiento generó las mejores recomendaciones para la entrada elegida? ¿Cómo implementaría una evaluación objetiva de estas recomendaciones? Justifique su respuesta.

(Utilice este espacio para describir su procedimiento)